# Missing Values Exercises

**Note**: 

> This exercise has been written out in something called a Jupyter Notebook. We'll discuss Jupyter Notebooks in more detail later in this specialization—they are very a powerful tool for data science communication!—but for the time being, the notebook is just a convenient way for us to write out the exercise. You don't need to *do* anything with the notebook except read its contents—just use write your Python code in a regular `.py` file.

**WARNING:**

> When asked to round your answers to a certain number of decimals, do *not* round any results until you've finished your computations and have your final answer! For example, if you were to calculate the average hourly wage for workers, and you did so by first calculating the average weekly salary of workers and the average hours worked per week, then divided the first number by the second, you should NOT round the average weekly salary of workers or the average hours worked per week. Rounding intermediate results can lead to compounding errors that cause problems for the autograder.


## Exercises

### Exercise 1

Today, we will be using the ACS data we used during out first `pandas` exercise to examine the US income distribution, and how it varies by race. Note that because the US income distribution has a very small number of people with *extremely* high incomes, and the ACS is just a sample of Americans, the far right tail of the distribution will not be very well estimated. However, this data should suffice for helping to understand wealth inequality in the United States. 

To begin, load the ACS Data in this exercise (the file is called `US_ACS_2017_10pct_sample.dta`).

In [1]:
import pandas as pd
import numpy as np

pd.set_option("mode.copy_on_write", True)


# Download the data
acs = pd.read_stata("data/US_ACS_2017_10pct_sample.dta")

In [2]:
# Too much output to be useful in a
# solution, but here's the code:
# for c in acs.columns:
#     print(c)

### Exercise 2

Let's begin by calculating the mean US incomes from this data (recall that income is stored in the `inctot` variable). 

Please round your answer to two decimal places.

In [3]:
results = dict()
results["ex2_avg_income"] = acs.inctot.mean()
print(
    f"The average value of `inctot` in the full ACS data is ${results['ex2_avg_income']:,.2f}"
)

The average value of `inctot` in the full ACS data is $1,723,646.27


### Exercise 3

Hmmm... That doesn't look right. The average American is definitely not earning that much a year! Let's look at the values of `inctot` using `value_counts()`. Do you see a problem?

Now use `value_counts()` with the argument `normalize=True` to see the proportion of the sample that reports each value instead of the count of people in each category. 

What share of our sample has an income of 9,999,999? Please round that value (already between 0 and 1) to three decimal places. 

What share has an income of 0? Please round that value (already between 0 and 1) to three decimal places.

In [4]:
shares = acs.inctot.value_counts(normalize=True)
results["ex3_share_making_9999999"] = shares.loc[9_999_999]
results["ex3_share_making_zero"] = shares.loc[0]
print(f"Share making 9,999,999 is: {results['ex3_share_making_9999999']:.3f}")
print(f"Share making zero income is: {results['ex3_share_making_zero']:.3f}")

Share making 9,999,999 is: 0.169
Share making zero income is: 0.106


### Exercise 4

As we discussed before, the ACS uses a value of 9999999 to denote that income information is not available for someone. The problem with using this kind of "sentinel value" is that pandas doesn't understand that this is supposed to denote missing data, and so when it averages the variable, it doesn't know to ignore 9999999. 

To help out `pandas`, use the `replace` command to replace all values of 9999999 with `np.nan`. 

In [5]:
acs["inctot"] = acs["inctot"].replace(9999999, np.nan)

### Exercise 5

Now that we've properly labeled our missing data as `np.nan`, let's calculate the average US income once more.

Please round your answer to two decimal places.

In [6]:
results["ex5_avg_income"] = acs.inctot.mean()
print(
    f"The average value of `inctot` in the ACS data after "
    f"replacing 9,999,999 with np.nan is {results['ex5_avg_income']:,.2f}"
)

The average value of `inctot` in the ACS data after replacing 9,999,999 with np.nan is 40,890.18


### Exercise 6

OK, now we've been able to get a reasonable average income number. As we can see, a major advantage of using `np.nan` is that `pandas` knows that `np.nan` observations should just be ignored when we are calculating means. 

But it's not enough to just get rid of the people who had `inctot` values of 9999999. We also need to know why those values were missing. Suppose, for example, that the value of 9999999 was used for anyone who made more than 100,000 dollars: if we just dropped those people, then our estimate of average income wouldn't mean much, would it?

So let's make sure we understand *why* data is missing for some people. If you recall from our last exercise, it seemed to be the case that most of the people who had incomes of 9999999 were children. Let's make sure that's true by looking at the distribution of the variable `age` for people for whom `inctot` is missing (i.e. subset the data to people with `inctot` missing, then look at the values of `age` with `value_counts()`).

Then do the opposite: look at the distribution of the `age` variable for people for whom `inctot` is *not* missing. 

Can you determine when 9999999 was being used? Is it ok we're excluding those people from our analysis?

Note: In this data, Python doesn't understand `age` is a number; it thinks it is a string because the original data has categories like "90 (90+ in 1980 and 1990)" and "less than 1 year old". So you can't just use `min()` or `max()`. We'll discuss converting string variables into numbers in a future class.

In [7]:
acs.loc[pd.isnull(acs.inctot), "age"].value_counts()

age
10    3997
9     3977
14    3847
12    3845
13    3800
      ... 
39       0
38       0
37       0
36       0
96       0
Name: count, Length: 97, dtype: int64

In [8]:
acs.loc[pd.notnull(acs.inctot), "age"].value_counts()

age
60                      4950
54                      4821
59                      4776
56                      4776
58                      4734
                        ... 
5                          0
4                          0
3                          0
2                          0
less than 1 year old       0
Name: count, Length: 97, dtype: int64

In [9]:
for i in acs.loc[pd.isnull(acs.inctot), "age"].unique().sort_values():
    print(i)

less than 1 year old
1
2
3
4
5
6
7
8
9
10
11
12
13
14


> Looks like the folks who are missing income are all very young, and everyone older has incomes. So must be that only people above a certain age were eligible to be asked their income.

### Exercise 7

Great, so now we know why those people had missing data, and we're ok with excluding them. 

But as we previously noted, there are also a lot of observations of zero income in our data, and it's not clear that we want everyone with a zero-income *should* be included in this average, since those may be people who are retired, or in school. 

Let's limit our attention to people who are currently working by subsetting to only employed respondents. We can do this using `empstat`. Remember you can use `value_counts()` to see what values of `empstat` are in the data!

How many observations are in your data once you subset for employed respondents?

In [10]:
# Check values taken by var
acs.empstat.value_counts()

empstat
employed              148758
not in labor force    104676
n/a                    57843
unemployed              7727
Name: count, dtype: int64

In [11]:
# now subset
employed = acs[acs.empstat == "employed"]
employed.shape

(148758, 104)

In [12]:
len(employed)

148758

### Exercise 8

Now let's estimate the racial income gap in the United States. 

What is the average salary for employed Black Americans? Please round your answer to two decimal places.

What is the average salary for employed White Americans?  Please round your answer to two decimal places.
 
How much more does the average White American make than the average Black American (in percentage terms)? (That is, what is the White average salary minus the Black average salary divided by the Black average salary time 100). You should get a number between 0 and 100. Please round it to one decimal place.

**Note:** these values are not quite accurate estimates. As we'll discuss in later lessons, to get completely accurate estimates from the ACS we have to take into account how people were selected to be interviewed. But you get pretty good estimates in most cases even without weights—your estimate of the racial wage gap without weights is within 5\% of the corrected value. 

**Note:** This is actually an underestimate of the wage gap. The US Census treats Hispanic respondents as a sub-category of "White." While all ethnic distinctions are socially constructed, and so on some level these distinctions are all deeply problematic, this coding is inconsistent with what most Americans think of when they hear the term "White," a term *most* Americans think of as a category that is mutually exclusive of being Hispanic or Latino (categories which are also usually conflated in American popular discussion). With that in mind, most researchers working with US Census data split "White" into "White, Hispanic" and "White, Non-Hispanic" using `race` *and* `hispan`. But for the moment, just identify "White" respondents using the value in `race`.


In [13]:
# Check values of `race`:
employed["race"].value_counts()

race
white                               116017
black/african american/negro         13175
other asian or pacific islander       6424
other race, nec                       5755
two major races                       3135
chinese                               2149
american indian or alaska native      1290
three or more major races              426
japanese                               387
Name: count, dtype: int64

In [14]:
# life is easier with simple race label
employed["race"] = employed.race.replace("black/african american/negro", "black")

In [15]:
# Avg Income
for r in ["white", "black"]:
    results[f"ex8_avg_income_{r}"] = employed.loc[
        (employed["race"] == r), "inctot"
    ].mean()

    print(
        f"Average income for employed {r.capitalize()} respondents "
        f"is ${results[f'ex8_avg_income_{r}']:,.2f}"
    )

Average income for employed White respondents is $60,473.15
Average income for employed Black respondents is $41,747.95


In [16]:
# Percent Whites make more than Blacks
results["ex8_racial_difference"] = (
    (results["ex8_avg_income_white"] - results["ex8_avg_income_black"])
    * 100
    / results["ex8_avg_income_black"]
)
print(
    f"Employed White Americans earns an average of {results['ex8_racial_difference']:.1f}% "
    "more than employed Black respondents."
)

Employed White Americans earns an average of 44.9% more than employed Black respondents.


### Exercise 9

As noted above, these estimates are not actually *quite* correct because we aren't taking into account the fact that when the US Census decided who to survey, not all Americans had the same likelihood of being asked. The US American Community Survey is an example of a *weighted* survey (essentially, people from smaller subpopulations have a higher likelihood of being included to ensure enough individuals in the final survey to constitute a representative sample that can be used statistically). 

To calculate a weighted average that takes into account these survey weights (in other words, a more accurate estimate of US incomes), you need to use the following formula:

$$weighted\_mean\_of\_x = \frac{\sum_i x_i * weight_i}{\sum_i weight_i}$$

(As you can see, when $weight_i$ is constant for all observations, this just simplifies to our normal formula for mean values. It is only when weights vary across individuals that weights must be explicitly addressed).

In this data, weights are stored in the variable `perwt`, which is the number of people for which each observation is a stand-in (the inverse of that observation's sampling probability). 

What is the *weighted* average salary for employed Black Americans? Please round your answer to two decimal places.

What is the *weighted* average salary for employed White Americans?  Please round your answer to two decimal places.


In [17]:
# Now the exact estimates taking into account sampling weights

for r in ["white", "black"]:
    group_employed = employed[employed["race"] == r]
    results[f"ex9_avg_income_{r}"] = (
        group_employed["inctot"]
        * group_employed["perwt"]
        / group_employed["perwt"].sum()
    ).sum()

    print(
        f"Weighted avg income for employed {r.capitalize()} Americans "
        f"is ${results[f'ex9_avg_income_{r}']:,.2f}"
    )

Weighted avg income for employed White Americans is $58,361.48
Weighted avg income for employed Black Americans is $40,430.95


### Exercise 10

Now calculate the weighted average income gap between employed *non-Hispanic* White Americans (see discussion in Exercise 8) and employed Black Americans. 

First, calculate the weighted average income for employed non-Hispanic White Americans. Please round your answer to two decimals.

How much more does the average non-Hispanic White American make than the average Black American (in percentage terms)? (That is, what is the non-Hispanic White average salary minus the Black average salary divided by the Black average salary). You should get a number between 0 and 100. Please round it to one decimal place.

In [18]:
white_nonhisp = employed.loc[
    (employed["race"] == "white") & (employed["hispan"] == "not hispanic"),
    ["inctot", "perwt"],
]

In [19]:
# White non-hispanic income
white_nonhisp_weighted = (
    white_nonhisp["inctot"] * white_nonhisp["perwt"]
).sum() / white_nonhisp["perwt"].sum()
white_nonhisp_weighted

61669.28927599297

In [20]:
results["ex10_wage_gap"] = (
    (white_nonhisp_weighted - results[f"ex9_avg_income_black"])
    * 100
    / results[f"ex9_avg_income_black"]
)

In [21]:
print(
    f"Employed non-Hispanic White Americans earns an average of "
    f"{results['ex10_wage_gap']:.1f}% "
    "more than employed Black respondents."
)

Employed non-Hispanic White Americans earns an average of 52.5% more than employed Black respondents.


### Exercise 11

Is that greater or less than the difference you found in Exercise 8? Why do you think that's the case?

> The gap is greater, which makes sense as White Hispanic Americans tend to work in less well-paid industries as compared to non-Hispanic White Americans. Thus their exclusion drives up the average calculated White wage, increasing the gap.

### Data Citation

The ACS data used in this exercise are a subsample of the IPUMS USA data available from [usa.ipums.org.](usa.ipums.org)

Please cite use of the data as follows: Steven Ruggles, Sarah Flood, Sophia Foster, Ronald Goeken, Jose Pacas, Megan Schouweiler and Matthew Sobek. IPUMS USA: Version 11.0 [dataset]. Minneapolis, MN: IPUMS, 2021. https://doi.org/10.18128/D010.V11.0

These data are intended for this exercise only. Individuals analyzing the data for other purposes must submit a separate data extract request directly via IPUMS USA.

Individuals are not to redistribute the data without permission. Contact ipums@umn.edu for redistribution requests.